# Getting Started: how to prepare data for Timelapse Feature Explorer

The [Timelapse Feature Explorer (TFE)](https://timelapse.allencell.org) is a web-based application designed for the interactive visualization and analysis of segmented time-series microscopy data! Data needs to be processed into a specific format to be loaded into the viewer.

In this tutorial, you'll learn how to prepare your data for the Timelapse Feature Explorer.

*This notebook can be run with Google Colab, but we recommend running it on a local machine if you plan to convert and view your own datasets later.*

<a target="_blank" href="https://colab.research.google.com/github/allen-cell-animated/colorizer-data/blob/main/documentation/getting_started_guide/GETTING_STARTED.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

## 1. Terms

A few key terms:

- **Dataset**: A dataset is a single time-series containing tracked objects and features. A dataset can contain up to about 16.7 million unique objects.
- **Raw dataset**: The raw data that you have collected or generated, before processing into the TFE format.
- **Collection**: An arbitrary grouping of datasets, allowing for easy comparison of multiple datasets in the viewer.
- **Object ID**: An ID associated with a single segmentation at a single timepoint. In the TFE-accepted format, object IDs must be sequential, starting from 0, and be unique across the whole dataset.
- **Track ID**: An identifier for a unique set of objects, linking their object IDs across timepoints. Generally this describes the track of one object along the time sequence.

## 2. Prerequisites

Setup instructions differ based on whether you are running a local Jupyter notebook instance or Google Colab.
> **_NOTE_**: You must be on Python version 3.9 or above. The installation may fail unexpectedly on older versions of Python.

#### 2.1A Running on a local machine with **Jupyter Lab**

From a command terminal, clone this repository if you haven't already and navigate to the `getting-started-guide` directory.
You will likely want to do this from a virtual Python environment. We've included steps below on how to activate a [venv](https://docs.python.org/3/library/venv.html) virtual Python environment.

```bash
# Skip this step if you've already cloned the repository.
git clone https://github.com/allen-cell-animated/colorizer-data.git
cd colorizer-data

# Set up a virtual environment
python -m venv venv
source .venv/bin/activate
# On Windows, you may need to run this instead:
# source .venv/Scripts/activate

# Install and start Jupyter lab
python -m pip install jupyterlab
jupyter lab
```

Follow the provided link to open the browser to access Jupyter, and navigate to this notebook in Jupyter Lab. Run the following cell to set up and install dependencies.

In [ ]:
# This may take a few minutes.
%pip install -r ./requirements.txt

#### 2.1B Running in **Google Colab**

If opening this notebook in Google Colab, run the following commands to install the repository and set up the environment.

In [ ]:
!git clone https://github.com/allen-cell-animated/colorizer-data.git

%cd colorizer-data/documentation/getting_started_guide

In [ ]:
%pip install -r ./requirements.txt

## 3. Expected formats for raw data

For this tutorial, we'll be working with sample data included in the [`getting_started_guide/raw_dataset`](./raw_dataset/) directory.

This dataset is a simplified example of raw, pre-processed segmentation data. The data was generated using the [`generate_raw_data.py` script](./scripts/generate_data.py), which generates a **CSV file** with columns for object IDs, track IDs, times, centroids, features (volume/height), and paths to the segmentation images. The **segmentation images** are 2D images in the OME-TIFF format, encoding the locations of segmented objects.

If your segmentation images are 3D, you may choose to flatten them to 2D or use our provided utilites to do so.

Your tracked object data may be in a different format, in which case it will need to be transformed to work well with our utilities. Generally, we recommend:

1. Save your data as a CSV or other format that can be read into a pandas `DataFrame`.
2. Make every segmented object its own row in the table.
3. Save track ID, time, centroids, and other information as columns.
4. Create columns for any additional features you want to visualize.

### What does the example dataset look like?

Here's a preview of the raw dataset, `data.csv`:

| object_id | track_id | time | centroid_x | centroid_y | area | radius | location | segmentation_path |
| ----------- | ---------- | ------ | ------------ | ------------ | -------- | -------- | ------------------- | --- |
| 1 | 0 | 0 | 33 | 110 | 706.9 | 15 | middle | frame_0.tiff |
| 2 | 1 | 0 | 67 | 100 | 804.2 | 16 | middle | frame_0.tiff |
| 3 | 2 | 0 | 100 | 108 | 804.2 | 16 | middle | frame_0.tiff |
| 4 | 3 | 0 | 133 | 88 | 706.9 | 15 | middle | frame_0.tiff |
| 5 | 4 | 0 | 167 | 101 | 804.2 | 16 | middle | frame_0.tiff |
| 6 | 0 | 1 | 33 | 121 | 530.9 | 13 | bottom | frame_1.tiff |
| 7 | 1 | 1 | 67 | 113 | 804.2 | 16 | middle | frame_1.tiff |
| ... | ... | ... | ... | ... | ... | ... | ... | ... |

Each of the segmentation images in the `segmentation_path` column is an OME-TIFF image containing the IDs of the segmented objects:

_Frame 0 of the example dataset, as viewed in FIJI. Contrast has been increased for easier viewing._

![Frame 0 of the example dataset, as viewed in FIJI. Contrast has been increased for easier viewing. The black background has been labeled as ID=0. Five red bubbles in the center of the image are labeled in ascending order from 1-5, left to right.](./assets/sample-segmentation.png)

> **_NOTE:_** A value of `0` is used to represent the background in the segmentation images. For simplicity, we recommend starting object IDs at `1` to avoid conflicts with the background value.

## 4. Processing data

Timelapse Feature Explorer reads data in the format specified by the [`DATA_FORMAT`](../DATA_FORMAT.md) document. We'll use the utilities provided by `colorizer-data` to convert to this format.

### Processing script

#### 1. Import dependencies and load the dataset into a pandas DataFrame

In [ ]:
from pathlib import Path

from bioio import BioImage
import pandas as pd

from colorizer_data.utils import (
    remap_segmented_image,
)
from colorizer_data.writer import (
    ColorizerDatasetWriter,
    ColorizerMetadata,
    FeatureInfo,
    FeatureType,
)

# Load the dataset
source_dataset_directory = Path("raw_dataset")
data: pd.DataFrame = pd.read_csv(source_dataset_directory / "data.csv")

#### 2. Configure the writer and data columns

In [ ]:
# Define column names as they appear in the loaded csv. Some of these are core data timelapse and others are the extra calculated features for use in the viewer.

# Core data columns
OBJECT_ID_COLUMN = "object_id"
TRACK_ID_COLUMN = "track_id"
TIMES_COLUMN = "time"
SEGMENTED_IMAGE_COLUMN = "segmentation_path"
CENTROIDS_X_COLUMN = "centroid_x"
CENTROIDS_Y_COLUMN = "centroid_y"

# Feature columns
AREA_COLUMN = "area"
LOCATION_COLUMN = "location"
RADIUS_COLUMN = "radius"

# Add in a column to act as an index for the dataset.
# This preserves row numbers even when the dataframe is grouped by
# time later.
INDEX_COLUMN = "index"
data = data.reset_index(drop=True)
data[INDEX_COLUMN] = data.index.values

# Create the writer, which we'll be using to save dataset-related files.
output_dir = "."
dataset_name = "processed_dataset"
writer = ColorizerDatasetWriter(output_dir, dataset_name)

#### 3. Write the core data

In [ ]:
# Turn core data columns into a numpy array, to be saved by the writer.
tracks = data[TRACK_ID_COLUMN].to_numpy()
times = data[TIMES_COLUMN].to_numpy()
centroids_x = data[CENTROIDS_X_COLUMN].to_numpy()
centroids_y = data[CENTROIDS_Y_COLUMN].to_numpy()

writer.write_data(
    tracks=tracks,
    times=times,
    centroids_x=centroids_x,
    centroids_y=centroids_y,
)

#### 4. Write the features

Features can be one of three types:

1. **Continuous** features are used for floating-point numbers.
2. **Discrete** features are used for integers.
3. **Categorical** features are used for string-based labels. Note that there can only be a max of 12 categories for a categorical feature.

The `FeatureInfo` class is used to provide metadata about each feature, such as its label, key, type, units, and descriptions if applicable.

In [ ]:
areas = data[AREA_COLUMN].to_numpy()
locations = data[LOCATION_COLUMN].to_numpy()
radii = data[RADIUS_COLUMN].to_numpy()

# Additional metadata can be provided for each feature, which will be shown
# when interacting with it in the viewer.
area_info = FeatureInfo(
    label="Area",
    key="area",
    type=FeatureType.CONTINUOUS,
    unit="px²",
    description="Area of object in square pixels, calculated from radius."
)
radius_info = FeatureInfo(
    label="Radius",
    key="radius",
    # Discrete features are used for integers.
    type=FeatureType.DISCRETE,
    unit="px",
    description="Radius of object in pixels."
)
location_info = FeatureInfo(
    label="Location",
    key="location",
    # Categorical features are used for string-based labels.
    type=FeatureType.CATEGORICAL,
    # Categories can be auto-detected from the data, or provided manually
    # if you want to preserve a specific order for the labels.
    categories=["top", "middle", "bottom"],
    description="Y position of object's centroid in the frame, as either 'top' (y < 40%), 'middle' (40% ≤ y ≤ 60%), or 'bottom' (y > 60%) of the frame."
)
writer.write_feature(areas, area_info)
writer.write_feature(radii, radius_info)
writer.write_feature(locations, location_info)

#### 5. Write the images

The `ColorizerDatasetWriter` needs to prepare images for fast visualization by converting them to PNGs with encoded object IDs. You can see more about what this looks like and how it works in our [data format documentation](../DATA_FORMAT.md#5-frames). As previously noted, all object IDs must be unique too, so this next section will perform three tasks:

1. Load in the image data from the segmentation images
2. Remap the object IDs to be unique across all timepoints
3. Write the remapped images using `ColorizerDatasetWriter.write_image()`

In [ ]:
# Group data by the timestamp
data_grouped_by_time = data.groupby(TIMES_COLUMN)
frame_paths = []

for frame_num, frame_data in data_grouped_by_time:
    # Get the path to the segmentation image and load it.
    frame_path = frame_data.iloc[0][SEGMENTED_IMAGE_COLUMN]
    segmentation_image = BioImage(source_dataset_directory / frame_path).get_image_data(
        "YX", S=0, T=0, C=0
    )
    # NOTE: For datasets with 3D segmentations, you may need to flatten the data into
    # 2D images. Typically, it's simplest to do so with a max projection, but may vary
    # based on your data. Replace the above line with the following:
    #
    # segmentation_image = bioio.BioImage(frame_path).get_image_data("ZYX", S=0, T=0, C=0)
    # segmentation_image = segmentation_image.max(axis=0)
    #
    # Remap the segmented so object IDs are unique across all timepoints.
    (remapped_segmentations, _lut) = remap_segmented_image(
        segmentation_image, frame_data, OBJECT_ID_COLUMN, INDEX_COLUMN
    )
    # Write the new segmentation image.
    frame_prefix = "frame_"
    frame_suffix = ".png"
    writer.write_image(remapped_segmentations, frame_num, frame_prefix, frame_suffix)
    image_path = frame_prefix + str(frame_num) + frame_suffix
    frame_paths.append(image_path)

writer.set_frame_paths(frame_paths)
print(frame_paths)

#### 6. Write the dataset and any additional metadata

In [ ]:
# Define the metadata for this dataset.
metadata = ColorizerMetadata(
    name="Example dataset",
    description="An example dataset for the Timelapse Feature Explorer.",
    author="Jane Doe et al.",
    dataset_version="v1.0",
    # The width and height of the original segmentations, in any arbitrary units.
    # This will control the scale bar in the viewer.
    frame_width=100,
    frame_height=100,
    frame_units="nm",
    # Time elapsed between each frame capture, in seconds.
    frame_duration_sec=1,
)

# Write the final dataset
writer.write_manifest(metadata=metadata)

Congratulations! If you got this far, you have a complete dataset ready to view in the TFE viewer. The new dataset will be found in the `processed_dataset` directory.

## 5. Viewing the dataset

Now that the dataset is processed, we can view it in the Timelapse Feature Explorer!

Timelapse Feature Explorer is designed to load datasets over the web using HTTP(S). This makes it easier to share datasets with collaborators and the public.

Your data must be hosted on a **web server** and served over **HTTPS** to be loaded on our public build. Alternatively, you can run a **local web server** and a **local instance** of the viewer. We'll cover steps for both options.

### 5.1 Viewing datasets via the web (HTTPS)

For this tutorial, you can load a pre-processed example copy of the dataset, which we've hosted on GitHub. You can access it at this URL: `https://raw.githubusercontent.com/allen-cell-animated/colorizer-data/main/documentation/getting_started_guide/example/processed_dataset/manifest.json`

If you have updated the dataset files or want to use a local dataset, skip to the next section.

1. Open Timelapse Feature Explorer at [https://timelapse.allencell.org](https://timelapse.allencell.org).

2. Click the **Load** in the header and paste in the following URL: `https://raw.githubusercontent.com/allen-cell-animated/colorizer-data/main/documentation/getting_started_guide/example/processed_dataset/manifest.json`

![The Load button on the Timelapse Feature Explorer header, next to the Help dropdown.](./assets/load-button.png)

> **_NOTE:_** You can either provide the URL of the directory containing a `manifest.json` or the full URL path of a `.json` file that follows the [manifest specification](../DATA_FORMAT.md#dataset). We recommend specifying the full URL path that includes the `manifest.json`.

Click **Load** in the popup menu to load the dataset. The viewer should appear with the dataset loaded!

### 5.2 Opening your dataset from a local directory over HTTP

To view a **locally converted dataset**, we'll also need to run a **local version** of the Timelapse Feature Explorer.

> ⚠ If you've been following along in Google Colab, this is the point where you'll want to switch to your local machine. Rerun the previous steps locally or download the processed dataset into `colorizer-data/documentation/getting_started_guide/processed_dataset` and continue from here.

<details>
<summary><b>[Why do we need to run a local instance of the viewer?]</b></summary>

--- 

TFE only loads data over the HTTP(S) web protocols, rather than directly loading local files. We can get around this by running a local server to serve our local files over HTTP.

However, our public build of TFE is served over HTTPS, a secure version of the HTTP protocol. For security reasons, HTTPS pages cannot load HTTP content. This means that the public version of TFE can only access other web content also hosted on HTTPS. 

It's difficult to set up an HTTPS server, so instead we can run TFE locally on HTTP! This will let it also access our local HTTP server.

![A diagram. The public version of TFE (HTTPS) can load files from a remote file server (HTTPS) but can't load it from a local file server (HTTP). A local version of TFE (HTTP) can load from both remote file servers and local file servers (HTTP and HTTPS).](./assets/http-diagram.png)

---
</details>

We've provided a simple script to start a local server and serve the processed dataset files. Open a command terminal in the `getting_started_guide` directory and run the following command:

```bash
python scripts/run_local_server.py 8080
```

In [ ]:
# If running this notebook locally (not in Colab), you can also run this code block to start the local server.
# Click stop in the toolbar to stop the server.
!python scripts/run_local_server.py 8080

Open your web browser and navigate to [`http://localhost:8080`](http://localhost:8080). You'll see a directory of files.

![A server webpage, titled "Directory listing for /". A list of files and directories follows. The "processed_dataset/" and "viewer/" directories are highlighted.](./assets/directory.png)

Navigate to the `processed_dataset` directory and right click on the **`manifest.json`** file. Select **Copy link address** to copy the URL of the file.

![A list of files as blue hyperlinks. The "manifest.json" file is highlighted and the context menu has appeared. The "Copy link address" option is hovered.](./assets/save-manifest.png)

Then, go back to the root directory of the file server and click the **`viewer/`** directory to open the viewer. Go to the **Load** button and paste in the copied URL of the `manifest.json` file to load the dataset.

![The Load button on the Timelapse Feature Explorer header, next to the Help dropdown.](./assets/load-button.png)

Once the dataset is open, click on the features dropdown to select a feature to visualize. You can also hover on individual objects to see their feature values in the viewport.

![The loaded dataset in Timelapse Feature Explorer. Five bubbles appear in the main viewport in various shades of purple and blue. The selected feature is Area, in units of pixels squared.](./assets/loaded-dataset.png)



## 6. What's next?

### Collections

If you have multiple datasets that you want to group together, you can create a **collection**. When loaded in the viewer, collections allows you to switch between any of the included datasets for easier comparison. See the [section on collections in our data format documentation](../DATA_FORMAT.md#collections) for more information.

Also see the `update_collection()` method in the [`utils.py` file](../../colorizer_data/utils.py) to create and update collections.

<details>
<summary><b>[🔍 Show me an example!]</b></summary>

---

Let's say we have two datasets that we want to group together in a collection. The directory structure would look like this:

```txt
📄 collection.json
📂 dataset_1/
  - 📄 manifest.json
  ...
📂 dataset_2/
  - 📄 manifest.json
  - ...
```

The `collection.json` file would look something like this:

`manifest.json:`

```txt
{
    "datasets": [
        { "name": "Dataset 1", "path": "dataset_1" },
        { "name": "Dataset 2", "path": "dataset_2" },
    ],
    "metadata": {
        "name": "My custom collection",
        "description": "Two example datasets grouped together for comparison",
        "author": "Jane Doe",
    }
}
```



---

</details>

### Advanced dataset conversion

We provide a number of example scripts that perform more advanced data processing tasks, such as:

1. Including bounding box data and outliers
2. Handling 3D segmentation images
3. Parallelizing frame processing
4. Grouping datasets into collections
5. Handling common arguments/options for processing scripts

All of these scripts are available in the [`bin/example_scripts` directory](../../colorizer_data/bin/example_scripts/).

### Installing Timelapse Feature Explorer locally
The pre-built version of TFE may not be up to date with the latest features and bugfixes. If you want to run your own local instance of TFE, you can clone the repository yourself and run it locally.

The repository for TFE and instructions on installation can be found at [https://github.com/allen-cell-animated/timelapse-colorizer](https://github.com/allen-cell-animated/timelapse-colorizer).
